# V3 Fraction Collector (Runze Valve)

In [1]:
from fraction_collector_v4 import FractionCollector
from sample_producer_flowreactor import SampleProducer
import threading
import queue
import time

In [2]:
# Initialize the FractionCollector
collector = FractionCollector(sensor_id=1, runze_valve_port='COM9', runze_valve_address=0, runze_valve_num_port=10, collection_num=3, waste_num=6)

Connected to CNC Machine!
Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']


In [8]:
collector.set_valve_state(3)

Current port set to 3


In [3]:
# Define well locations
loc = 'well_plate_location'
#loc = '96_well_plate_location'

In [4]:
# Shared queue and event
sample_queue = queue.Queue()
stop_event = threading.Event()

# Initialize producer
producer = SampleProducer(
    reaction_profile_csv='SL_1.csv',
    sample_queue=sample_queue,
    stop_event=stop_event,
    tubing_length_m=1.0,
    tubing_id_inch=0.03,
    buffer_factor=1.2
)
producer.start()

[12:03:07] === Sample Collection Summary ===
[12:03:07] Reaction name         : 20251027_SL1
[12:03:07] Waste time (s)        : 1398.7
[12:03:07] Collection time (s)   : 1382.4
[12:03:07] Flow rate (mL/min)    : 0.50
[12:03:07] Tubing length (m)     : 1.0
[12:03:07] Tubing ID (inch)      : 0.03
[12:03:07] Buffer factor         : 1.2
[12:03:07] Tubing delay (s)      : 54.7
[12:03:07] Adjusted waste time   : 1464.4 s
[12:03:07] Adjusted collect time : 1152.0 s
[12:03:07] =================================
[12:03:07] Starting waste phase for 1464.4 seconds...


[12:03:37] Wasting... 30 / 1464 seconds elapsed
[12:04:07] Wasting... 60 / 1464 seconds elapsed
[12:04:37] Wasting... 90 / 1464 seconds elapsed
[12:05:07] Wasting... 120 / 1464 seconds elapsed
[12:05:37] Wasting... 150 / 1464 seconds elapsed
[12:06:07] Wasting... 180 / 1464 seconds elapsed
[12:06:37] Wasting... 210 / 1464 seconds elapsed
[12:07:07] Wasting... 240 / 1464 seconds elapsed
[12:07:37] Wasting... 270 / 1464 seconds elapsed
[12:08:07] Wasting... 300 / 1464 seconds elapsed
[12:08:37] Wasting... 330 / 1464 seconds elapsed
[12:09:07] Wasting... 360 / 1464 seconds elapsed
[12:09:37] Wasting... 390 / 1464 seconds elapsed
[12:10:07] Wasting... 420 / 1464 seconds elapsed
[12:10:37] Wasting... 450 / 1464 seconds elapsed
[12:11:07] Wasting... 480 / 1464 seconds elapsed
[12:11:37] Wasting... 510 / 1464 seconds elapsed
[12:12:07] Wasting... 540 / 1464 seconds elapsed
[12:12:37] Wasting... 570 / 1464 seconds elapsed
[12:13:07] Wasting... 600 / 1464 seconds elapsed
[12:13:37] Wasting... 6

In [5]:
# One Sample Multiple Fractions
number_of_samples = 3
loc_index = 0
loc_threshold = loc_index + number_of_samples
collection_thread = None

while not stop_event.is_set():
    try:
        sample = sample_queue.get(timeout=0.1)
        if collection_thread and collection_thread.is_alive():
            print("❌ Not enough time between samples — shutting down.")
            stop_event.set()
            break
        for i in range(number_of_samples): # Collect 3 fractions per sample
            #collection_thread = threading.Thread(target=collect_sample_sim, args=(sample, loc_index, 3))
            collection_thread = threading.Thread(target=collector.collect_fraction, args=(140, loc, loc_index, 20)) #real collection
            collection_thread.start()
            collection_thread.join()
            loc_index += 1

    except queue.Empty:
        pass

    if loc_index >= loc_threshold:  # match number of samples
        break

# Wait for last collection to finish
if collection_thread and collection_thread.is_alive():
    collection_thread.join()

# Final result
if stop_event.is_set() or loc_index < loc_threshold:
    print("❌ Not enough samples")
else:
    print("✅ All samples collected successfully.")

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 20 drops...

Current port set to 3
Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Current port set to 6
Rinsing complete.

Moving to location: well_plate_location at index 0
Movement commands rendered: 4
Moved safely To (X35.5, Y132, Z-12):  ['ok']

Current port set to 3
Starting fraction collection at well_plate_location (index 0)...

Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
Timeout reached.
Failed to collect enough drops at well_plate_location (index 0)...

Current port set to 6
Moving to location: cnc_waste_location at index 0
Mov

In [ ]:
# Multiple Samples
loc_index = 0
loc_threshold = loc_index + number_of_samples
collection_thread = None

while not stop_event.is_set():
    try:
        sample = sample_queue.get(timeout=0.1)
        if collection_thread and collection_thread.is_alive():
            print("❌ Not enough time between samples — shutting down.")
            stop_event.set()
            break

        #collection_thread = threading.Thread(target=collect_sample_sim, args=(sample, loc_index, 3))
        collection_thread = threading.Thread(target=collector.collect_fraction, args=(70, loc, loc_index, 20)) #real collection
        collection_thread.start()
        loc_index += 1

    except queue.Empty:
        pass

    if loc_index >= loc_threshold:  # match number of samples
        break

# Wait for last collection to finish
if collection_thread and collection_thread.is_alive():
    collection_thread.join()

# Final result
if stop_event.is_set() or loc_index < loc_threshold:
    print("❌ Not enough samples")
else:
    print("✅ All samples collected successfully.")

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 20 drops...

Current port set to 3
Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Current port set to 6
Rinsing complete.

Moving to location: well_plate_location at index 0
Movement commands rendered: 4
Moved safely To (X35.5, Y129.5, Z-12):  ['ok']

Current port set to 3
Starting fraction collection at well_plate_location (index 0)...

Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Fraction collection of well_plate_location (index 0) complete.

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
✅ All samples collected successfully.


In [5]:
stop_event.set()